Membuat model random forest dan preprocessing

In [1]:
import joblib

# load the trained model and preprocessor
rf_model = joblib.load('random_forest_model.pkl')
preprocess = joblib.load('prioprocessor.pkl')


Uji pada sampel inferensi pertama

In [2]:
import pandas as pd

# 1. create a sampel input row (use real values later)
sample_input = {
    'warehouse_block': 'F',
    'mode_of_shipment': 'Flight',
    'customer_care_calls': 4,
    'customer_rating': 3,
    'cost_of_the_product': 250,
    'prior_purchases': 2,
    'product_importance': 'low',
    'gender': 'M',
    'discount_offered': 10,
    'weight_category': 'medium'
}

# 2. convert to a dataframe
df_sample = pd.DataFrame([sample_input])

# 3. preprocess the sampel
df_sample_preprocessed = preprocess.transform(df_sample)

# 4. prediksi using the loaded model
prediction = rf_model.predict(df_sample_preprocessed)[0]

# 5. display result
result = "Late Delivery (1)" if prediction == 1 else "On Time (0)"
print("Predicted:", result)


Predicted: Late Delivery (1)


Model berhasil memprediksi dengan data sample

Mari uji pada data frame asli dari x_test

In [3]:
import joblib

# load uji data
X_test = joblib.load('X_test.pkl')
y_test = joblib.load('y_test.pkl')


In [4]:
# preprocess the full uji set
X_test_preprocessed = preprocess.transform(X_test)

# prediksi all at once
y_pred = rf_model.predict(X_test_preprocessed)

# optional: show performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=["On Time (0)", "Late (1)"]))


              precision    recall  f1-score   support

 On Time (0)       0.55      0.65      0.60       900
    Late (1)       0.72      0.63      0.68      1300

    accuracy                           0.64      2200
   macro avg       0.64      0.64      0.64      2200
weighted avg       0.65      0.64      0.64      2200



In [5]:
# select 3 real rows from x_uji
sample_df = X_test.iloc[:3]  # change the range if you want different rows

# preprocess
sample_preprocessed = preprocess.transform(sample_df)

# prediksi
predictions = rf_model.predict(sample_preprocessed)

# display results with real inputs
for i, pred in enumerate(predictions):
    label = "Late Delivery (1)" if pred == 1 else "On Time (0)"
    print(f"Sample {i+1} Prediction: {label}")


Sample 1 Prediction: Late Delivery (1)
Sample 2 Prediction: Late Delivery (1)
Sample 3 Prediction: Late Delivery (1)


Selanjutnya membandingkan Actual dan Predicted

In [ ]:
# compare true vs prediction
pd.DataFrame({
    "Actual": y_test.iloc[:3].values,
    "Predicted": predictions
})


,Actual,Predicted
0,1,1
1,1,1
2,0,1


Model berhasil memprediksi 2 sample delay namun gagar dalam meprediksi sample 2 yang harusnya ontime

In [7]:
from sklearn.metrics import classification_report

X_test_preprocessed = preprocess.transform(X_test)
y_pred = rf_model.predict(X_test_preprocessed)

print(classification_report(y_test, y_pred, target_names=["On Time (0)", "Late (1)"]))


              precision    recall  f1-score   support

 On Time (0)       0.55      0.65      0.60       900
    Late (1)       0.72      0.63      0.68      1300

    accuracy                           0.64      2200
   macro avg       0.64      0.64      0.64      2200
weighted avg       0.65      0.64      0.64      2200



Karena model gagal memprediksi mari kita cek confusion matrixnya untuk melihat seberapa baik model memprediksi false negative dan false positive

In [8]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))


[[586 314]
 [477 823]]


586 True Positives (Berhasil memprediksi "On Time")

823 True Negatives (Berhasil memprediksi "delay")

314 False Positives (Memprediksi delay, tapi sebenarnya on time) → false alarm

477 False Negatives (Memprediksi On Time, tapi sebenarnya delay) → ⚠️ risky



Model masih buruk untuk mendeteksi false negative. Mari kita ubah threshold untuk melihat apakah bisa ada perbaikan untuk recall sehingga bisa memaksimalkan pendeteksian delay.

kita mencoba beberapa threshold random forest dan memutuskan untuk menggunakan 0.4 untuk memaksimalkan model mendeteksi delay walaupun dengan resiko memperburuk false positive.


In [9]:
# get prediksiion probabilities
y_proba = rf_model.predict_proba(X_test_preprocessed)[:, 1]

# prediksi with a lower threshold (e.g. 0.4 instead of 0.5)
y_pred_thresh = (y_proba >= 0.4).astype(int)

# evaluate
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred_thresh))
print(classification_report(y_test, y_pred_thresh, target_names=["On Time (0)", "Late (1)"]))


[[ 315  585]
 [ 251 1049]]
              precision    recall  f1-score   support

 On Time (0)       0.56      0.35      0.43       900
    Late (1)       0.64      0.81      0.72      1300

    accuracy                           0.62      2200
   macro avg       0.60      0.58      0.57      2200
weighted avg       0.61      0.62      0.60      2200



Hasilnya ketika diubah menjadi threshold di 0.4 kita bisa mendapatkan model yang cukup baik di recall sebesar 0.81

Model tersebut berhasil meningkatkan kemampuannya untuk mendeteksi keterlambatan pengiriman dengan mengurangi hasil false negative dari 477 menjadi 251 — hampir menguranginya hingga setengahnya. Hal ini menunjukkan bahwa kami telah membangun model yang sangat mampu mengidentifikasi potensi keterlambatan, sebagaimana tercermin dalam kinerja penarikan kembali yang kuat.

In [10]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    precision_recall_curve,
    roc_curve,
    roc_auc_score,
    average_precision_score
)

def plot_threshold_analysis(model, X, y):
    # prediksi probabilities
    y_proba = model.predict_proba(X)[:, 1]

    # precision-recall curve
    precision, recall, thresholds_pr = precision_recall_curve(y, y_proba)

    # roc curve
    fpr, tpr, thresholds_roc = roc_curve(y, y_proba)
    auc_score = roc_auc_score(y, y_proba)
    ap_score = average_precision_score(y, y_proba)

    # plot
    plt.figure(figsize=(14, 5))

    # --- precision-recall ---
    plt.subplot(1, 2, 1)
    plt.plot(recall, precision, marker='.', label=f'AP = {ap_score:.2f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.grid(True)
    plt.legend()

    # --- roc ---
    plt.subplot(1, 2, 2)
    plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
    plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()


## Conclusion

Proyek ini berhasil mengembangkan model prediksi keterlambatan pengiriman dengan memanfaatkan data historis pelanggan dan pengiriman. Dari beberapa algoritma yang diuji, **Random Forest Classifier** terbukti paling efektif dalam menangani berbagai tipe data dan menangkap interaksi kompleks antar fitur.

Untuk menyelaraskan performa model dengan kebutuhan bisnis—yaitu meminimalkan terlewatnya deteksi pengiriman terlambat—kami menyesuaikan ambang klasifikasi dari **0.50 menjadi 0.40**. Hasilnya:

- **Recall meningkat** untuk kelas terlambat (Late) dari **0.63 menjadi 0.81**  
- **False negatives berkurang** hampir 50% (dari 477 menjadi 251)  
- **False positives** memang bertambah, namun ini trade-off yang dapat diterima untuk deteksi dini risiko operasional

### Kelebihan Model
- **Recall tinggi** pada pengiriman terlambat  
- **Stabil** di berbagai tipe dan distribusi fitur  
- **Mudah diinterpretasi** dan cepat diimplementasikan untuk penggunaan real-time

### Keterbatasan Model
- **False positive** lebih banyak pada ambang 0.40—mungkin memerlukan validasi manual  
- Bergantung pada **kualitas data historis**; akurasi bisa menurun tanpa retraining  

### Manfaat Bisnis

Model ini dirancang khusus untuk **UKM logistik** yang memerlukan solusi ringan namun andal. Dengan implementasi berbasis web (Streamlit + Hugging Face), tim non-teknis dapat:

- **Memantau risiko keterlambatan** secara proaktif  
- **Mengurangi dampak operasional** melalui peringatan dini  
- **Meningkatkan kepercayaan pelanggan** tanpa investasi besar  

Dengan demikian, model ini siap digunakan sebagai alat bantu pengambilan keputusan sehari-hari dalam mengelola pengiriman dan menjaga reputasi layanan.
